In [1]:
import time
import keras

from keras.models import Model
from keras.layers import Dropout, Flatten, BatchNormalization, TimeDistributed, Input, Add, Concatenate
from keras.layers import Dense, Conv2D, MaxPooling2D, LSTM, TimeDistributed, Reshape
import keras.backend as K
import keras.callbacks as callbacks

import pandas as pd
import numpy as np
from numpy import array
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

2023-04-19 12:35:14.377485: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:

save_path = "/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/"
model_name = "conv_only_6"
kf = KFold(n_splits = 10, shuffle =True)
num_epochs = 10

#### define functions ####

def model_cycle():
    inputs = Input(shape=(50, 4, 1))
        
    x = Conv2D(48, kernel_size=(21,4),
                   activation='relu',
                   padding='valid')(inputs)
    x = MaxPooling2D((2,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    # print(f"After first convolutional layer: x.shape = {x.shape}")

    # x = Reshape((K.int_shape(x)[1], K.int_shape(x)[3]))(x)
    # x = Conv1D(48, kernel_size=(11),
    #                activation='relu',
    #                padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)

    # print(f"After second convolutional layer: x.shape = {x.shape}")

    # x = Conv2D(48, kernel_size=(21,1),
    #                activation='relu',
    #                padding='same')(x)
    # x = MaxPooling2D((24,1),padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)

    # print(f"After third convolutional layer, x.shape = {x.shape}")

    # x = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)
    # x = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(x)
    # x = MaxPooling2D((12,1),padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)

    # # parallel line 1
    # fx1 = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(x)
    # fx1 = BatchNormalization()(fx1)
    # fx1 = Dropout(0.2)(fx1)
    # fx1 = Conv2D(48, kernel_size=(3,1),
    #                activation='relu',
    #                padding='same')(fx1)
    # fx1 = MaxPooling2D((2,1),padding='same')(fx1)
    # fx1 = BatchNormalization()(fx1)
    # fx1 = Dropout(0.2)(fx1)
    
    # # parallel line 2
    # fx2 = Conv2D(48, kernel_size=(11,1),
    #                activation='relu',
    #                padding='same')(x)
    # fx2 = BatchNormalization()(fx2)
    # fx2 = Dropout(0.2)(fx2)
    # fx2 = Conv2D(48, kernel_size=(21,1),
    #                activation='relu',
    #                padding='same')(fx2)
    # fx2 = MaxPooling2D((2,1),padding='same')(fx2)
    # fx2 = BatchNormalization()(fx2)
    # fx2 = Dropout(0.2)(fx2)
    
    # # # Add
    # x1 = Concatenate(axis=-3)([fx1, fx2])
    # x = Add()([x, x1])
    # x = MaxPooling2D((2,1),padding='same')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.2)(x)
    
    # x = Reshape((K.int_shape(x)[1], K.int_shape(x)[3]))(x)
    # x = LSTM(20, return_sequences=False)(x)
    # x = Dropout(0.2)(x)

    # x = Reshape((1, 24*48))(x)
    x = Flatten()(x)

    outputs = Dense(1, activation='linear')(x)
    # print(outputs.shape)
    network = Model(inputs, outputs)
    network.compile(optimizer='rmsprop',
                    loss='mean_squared_error')
    return network
    
def dnaOneHot(sequence):
    seq_array = array(list(sequence))
    code = {"A": [0], "C": [1], "G": [2], "T": [3], "N": [4],
            "a": [0], "c": [1], "g": [2], "t": [3], "n": [4]}
    onehot_encoded_seq = []
    for char in seq_array:
        onehot_encoded = np.zeros(5)
        onehot_encoded[code[char]] = 1
        onehot_encoded_seq.append(onehot_encoded[0:4])
    return onehot_encoded_seq



In [12]:
def display_fits(fits):
    print(f"Average correlation on tiling: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 5])}",
          f"\nAverage MSE on tiling: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 5])}",
          f"\nAverage correlation on random: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 3])}",
          f"\nAverage MSE on random: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 3])}",
          f"\nAverage correlation on ChrV: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 7])}",
          f"\nAverage MSE on ChrV: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 7])}",
          f"\nAverage correlation on CN: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 8) == 0])}",
          f"\nAverage MSE on CN: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 8) == 0])}")

In [13]:
def find_c0new(dat):
  mat = np.empty((3,3), float)
  k = 2*np.pi/10.4
  n = array([26, 29, 31])
  mat[0:3,0] = 1
  mat[0:3, 1] = np.sin(n*k)
  mat[0:3, 2] = np.cos(n*k)
  inv_mat = np.linalg.inv(mat)
  c0A1A2 = array(np.matmul(dat[["n=26", "n=29", "n=31"]], np.transpose(inv_mat)))
  c0Aphi = c0A1A2
  c0Aphi[:,0] = c0A1A2[:,0]
  c0Aphi[:,1] = np.sqrt(c0A1A2[:,1]**2 + c0A1A2[:,2]**2)
  c0Aphi[:,2] <- np.sign(c0A1A2[:,2]) * np.arccos(c0A1A2[:,1]/c0Aphi[:,1])
  return c0Aphi[:,0]

In [6]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.process_time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.process_time() - self.epoch_time_start)
        
#### preparing data ####

data_cerevisiae_nucle = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle1.txt",delimiter = ",")
X1 = []
for sequence_nt in data_cerevisiae_nucle["Sequence"]:
    X1.append(dnaOneHot(sequence_nt))
X1 = array(X1)
X1 = X1.reshape((X1.shape[0],50,4,1))
X1_reverse = np.flip(X1,[1,2])
# Y1 = data_cerevisiae_nucle["C0"].values.astype(float)
Y1 = find_c0new(data_cerevisiae_nucle).astype(float)


In [7]:
data_random_library = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle3.txt",delimiter = ",")
X3 = []
for sequence_nt in data_random_library["Sequence"]:
    X3.append(dnaOneHot(sequence_nt))
X3 = array(X3)
X3 = X3.reshape((X3.shape[0],50,4,1))
X3_reverse = np.flip(X3,[1,2])
# Y3 = data_random_library["C0"].values.astype(float)
Y3 = find_c0new(data_random_library).astype(float)


In [8]:
data_tiling = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle5.txt",delimiter = ",")
X5 = []
for sequence_nt in data_tiling["Sequence"]:
    X5.append(dnaOneHot(sequence_nt))
X5 = array(X5)
X5 = X5.reshape((X5.shape[0],50,4,1))
X5_reverse = np.flip(X5,[1,2])
# Y5 = data_tiling["C0"].values.astype(float)
Y5 = find_c0new(data_tiling).astype(float)



In [9]:
data_chr5 = pd.read_csv("/Users/Brody1/Documents/Northwestern/DNA_Cyclizability/cycle6.txt",delimiter = ",")
X6 = []
for sequence_nt in data_chr5["Sequence"]:
    X6.append(dnaOneHot(sequence_nt))
X6 = array(X6)
X6 = X6.reshape((X6.shape[0],50,4,1))
X6_reverse = np.flip(X6,[1,2])
# Y6 = data_chr5["C0"].values.astype(float)
Y6 = find_c0new(data_chr5).astype(float)

In [14]:
#### tiling

VALIDATION_LOSS = []
fold_var = 1
n = Y5.shape[0]

fits = []
detrend = []
times = []
times2 = []

for train_index, val_index in kf.split(Y5):
    training_X = X5[train_index]
    training_X_reverse = X5_reverse[train_index]
    validation_X = X5[val_index]
    validation_X_reverse = X5_reverse[val_index]
    training_Y = Y5[train_index]
    validation_Y = Y5[val_index]
    # CREATE NEW MODEL
    model = model_cycle()
    # CREATE CALLBACKS
    checkpoint = callbacks.ModelCheckpoint(save_path + model_name+"_tiling_"+str(fold_var)+".h5",
                                                    monitor='val_loss', verbose=1,
                                                    save_best_only=True, mode='min')
    time_callback = TimeHistory()

    history = model.fit(training_X, training_Y,
                        epochs=num_epochs,
                        callbacks= [checkpoint, time_callback],
                        validation_data=(validation_X, validation_Y))
    model.load_weights(save_path + model_name+"_tiling_"+str(fold_var)+".h5")
    model.save(save_path+model_name+"_tiling_"+str(fold_var),save_traces=False)
    times.append(time_callback.times)

    pred_Y = model.predict(training_X)
    pred_Y = pred_Y.reshape(pred_Y.shape[0])
    pred_Y_reverse = model.predict(training_X_reverse)
    pred_Y_reverse = pred_Y_reverse.reshape(pred_Y_reverse.shape[0])
    pred_Y = (pred_Y+pred_Y_reverse)/2
    reg =  LinearRegression().fit(array(pred_Y).reshape(-1, 1), array(training_Y).reshape(-1, 1))
    
    detrend_int = reg.intercept_
    detrend_slope = reg.coef_
    detrend.append([float(detrend_int), float(detrend_slope)])

    start_time = time.process_time()
    fit = model.predict(X1)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X1_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X3)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X3_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(validation_X)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(validation_X_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    start_time = time.process_time()
    fit = model.predict(X6)
    fit = fit.reshape(fit.shape[0])
    fit_reverse = model.predict(X6_reverse)
    fit_reverse = fit_reverse.reshape(fit_reverse.shape[0])
    reverse_corr = np.corrcoef(fit, fit_reverse)[0,1]
    fit = (fit + fit_reverse)/2
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit),reverse_corr]
    fits.append(fit_tmp)
    fit = detrend_int + fit * detrend_slope
    fit = fit.flatten()
    fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit),reverse_corr]
    time0 = time.process_time() - start_time
    times2.append([time0])
    fits.append(fit_tmp)
    
    K.clear_session()
    fold_var += 1
    
detrend = array(detrend)
detrend = pd.DataFrame(detrend)
detrend.to_csv(save_path +model_name+"_detrend_tiling.txt", index = False)

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+ "_fits_tiling.txt", index = False)

with open(save_path +model_name+"_time_tiling.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')

with open(save_path +model_name+"_pred_time_tiling.txt", "w") as file:
    for row in times2:
        s = " ".join(map(str, row))
        file.write(s+'\n')


Epoch 1/10
2312/2317 [============================>.] - ETA: 0s - loss: 0.2401
Epoch 1: val_loss improved from inf to 0.16885, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_1.h5
2317/2317 [==============================] - 6s 3ms/step - loss: 0.2400 - val_loss: 0.1689
Epoch 2/10
2306/2317 [============================>.] - ETA: 0s - loss: 0.1678
Epoch 2: val_loss improved from 0.16885 to 0.15421, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_1.h5
2317/2317 [==============================] - 5s 2ms/step - loss: 0.1679 - val_loss: 0.1542
Epoch 3/10
2307/2317 [============================>.] - ETA: 0s - loss: 0.1605
Epoch 3: val_loss did not improve from 0.15421
2317/2317 [==============================] - 6s 3ms/step - loss: 0.1605 - val_loss: 0.1571
Epoch 4/10
2308/2317 [============================>.] - ETA: 0s - loss: 0.1567
Epoch 4: val_loss improv

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_1/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_1/assets


2576/2576 [==============================] - 2s 768us/step
Epoch 1/10
2303/2317 [============================>.] - ETA: 0s - loss: 0.2401
Epoch 1: val_loss improved from inf to 0.17788, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_2.h5
2317/2317 [==============================] - 6s 2ms/step - loss: 0.2397 - val_loss: 0.1779
Epoch 2/10
2310/2317 [============================>.] - ETA: 0s - loss: 0.1675
Epoch 2: val_loss improved from 0.17788 to 0.16024, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_2.h5
2317/2317 [==============================] - 6s 2ms/step - loss: 0.1675 - val_loss: 0.1602
Epoch 3/10
2316/2317 [============================>.] - ETA: 0s - loss: 0.1585
Epoch 3: val_loss improved from 0.16024 to 0.15105, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_2.h5
2317/2317 

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_2/assets


...

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_9/assets


2576/2576 [==============================] - 2s 781us/step
Epoch 1/10
2313/2317 [============================>.] - ETA: 0s - loss: 0.2345
Epoch 1: val_loss improved from inf to 0.16373, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_10.h5
2317/2317 [==============================] - 6s 3ms/step - loss: 0.2345 - val_loss: 0.1637
Epoch 2/10
2313/2317 [============================>.] - ETA: 0s - loss: 0.1659
Epoch 2: val_loss improved from 0.16373 to 0.15348, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_10.h5
2317/2317 [==============================] - 6s 2ms/step - loss: 0.1659 - val_loss: 0.1535
Epoch 3/10
2302/2317 [============================>.] - ETA: 0s - loss: 0.1605
Epoch 3: val_loss improved from 0.15348 to 0.14567, saving model to /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_10.h5
2317/23

INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_10/assets


INFO:tensorflow:Assets written to: /Users/Brody1/Documents/Northwestern/DNA_Cyclizability/benchmarks/deep-learning/conv_only_6_tiling_10/assets


2576/2576 [==============================] - 3s 965us/step


In [15]:
display_fits(fits)

Average correlation on tiling: 0.6590394330352688 
Average MSE on tiling: 0.13701102262638046 
Average correlation on random: 0.638575133606921 
Average MSE on random: 0.0861782779830948 
Average correlation on ChrV: 0.5408828757990819 
Average MSE on ChrV: 0.20603591560645845 
Average correlation on CN: 0.6125182092046465 
Average MSE on CN: 0.1366847946233453
